# Item Based Collobrative Filtering

An online movie viewing platform wants to develop a recommendation system with collaborative filtering method. The company, which has tried content-based recommendation systems, wants to develop recommendations in a way that accommodates the opinions of the community. When users like a movie, other movies that have a **similar liking pattern** with that movie are requested to be suggested.

The dataset is provided by [**MovieLens**](https://grouplens.org/datasets/movielens/)
. 

In [19]:
# libraries 
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)

sns.set(rc={"figure.figsize":(12,12)})

## Dataset Preparation

In [20]:
def create_user_df():
    
    movie = pd.read_csv("datas/movie.csv")
    rating = pd.read_csv("datas/rating.csv")
    
    # merging the data
    data = movie.merge(rating, how="left",on="movieId")
    
    comment_counts = pd.DataFrame(data["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 10000].index # select less rated movies
    common_movies = data[~data["title"].isin(rare_movies)] # the movies which are not less rates
    
    user_movie_df = common_movies.pivot_table(index=["userId"],columns=["title"],values="rating")
    
    return user_movie_df

In [21]:
user_movie_df = create_user_df()
user_movie_df = user_movie_df.astype(np.float32)
user_movie_df.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Airplane! (1980),Aladdin (1992),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),"American President, The (1995)",Animal House (1978),Annie Hall (1977),Antz (1998),Apocalypse Now (1979),Apollo 13 (1995),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),As Good as It Gets (1997),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Back to the Future Part III (1990),Bad Boys (1995),Basic Instinct (1992),Batman (1989),Batman Begins (2005),Batman Forever (1995),Batman Returns (1992),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),Best in Show (2000),Beverly Hills Cop (1984),Beverly Hills Cop III (1994),Big (1988),Big Fish (2003),"Big Lebowski, The (1998)","Birdcage, The (1996)","Birds, The (1963)",Black Hawk Down (2001),Blade (1998),Blade Runner (1982),"Blair Witch Project, The (1999)",Blazing Saddles (1974),"Blues Brothers, The (1980)",Boogie Nights (1997),"Boot, Das (Boat, The) (1981)","Bourne Identity, The (2002)","Bourne Supremacy, The (2004)","Bourne Ultimatum, The (2007)",Bowling for Columbine (2002),Braveheart (1995),Brazil (1985),"Breakfast Club, The (1985)","Bridge on the River Kwai, The (1957)",Bridget Jones's Diary (2001),Broken Arrow (1996),"Bug's Life, A (1998)",Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Casablanca (1942),Casino (1995),Casino Royale (2006),Casper (1995),Cast Away (2000),Catch Me If You Can (2002),Charlie's Angels (2000),Chasing Amy (1997),Chicken Run (2000),Chinatown (1974),Chocolat (2000),"Christmas Story, A (1983)",Citizen Kane (1941),City Slickers II: The Legend of Curly's Gold (1994),City of God (Cidade de Deus) (2002),Clear and Present Danger (1994),Clerks (1994),"Client, The (1994)",Cliffhanger (1993),"Clockwork Orange, A (1971)",Close Encounters of the Third Kind (1977),Clueless (1995),Cocoon (1985),Con Air (1997),Coneheads (1993),Congo (1995),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Crimson Tide (1995),Crocodile Dundee (1986),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)","Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dark City (1998),"Dark Knight, The (2008)",Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),Deep Impact (1998),"Deer Hunter, The (1978)",Demolition Man (1993),"Departed, The (2006)",Desperado (1995),"Devil's Advocate, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Dirty Dancing (1987),Disclosure (1994),Dogma (1999),Donnie Brasco (1997),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dracula (Bram Stoker's Dracula) (1992),Dragonheart (1996),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),Edward Scissorhands (1990),Election (1999),Enemy of the State (1998),"English Patient, The (1996)",Eraser (1996),Erin Brockovich (2000),Eternal Sunshine of the Spotless Mind (2004),Executive Decision (1996),"Exorcist, The (1973)",Eyes Wide Shut (1999),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Ferris Bueller's Day Off (1986),"Few Good Men, A (1992)",Field of Dreams (1989),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)",First Knight (1995),"Fish Called Wanda, A (1988)"

In [22]:
user_movie_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Float64Index: 137658 entries, 1.0 to 138493.0
Columns: 462 entries, 10 Things I Hate About You (1999) to Young Frankenstein (1974)
dtypes: float32(462)
memory usage: 243.7 MB


In [23]:
len(user_movie_df)

137658

In [24]:
def item_based_recommender(movie_name:str, user_movie_df):
    
    movie_name = user_movie_df[movie_name]
    
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

In [25]:
item_based_recommender("Schindler's List (1993)",user_movie_df)

title
Schindler's List (1993)                       1.000000
Saving Private Ryan (1998)                    0.438793
Pianist, The (2002)                           0.370002
Philadelphia (1993)                           0.362108
Gandhi (1982)                                 0.345513
Rain Man (1988)                               0.345187
Shawshank Redemption, The (1994)              0.338183
Life Is Beautiful (La Vita è bella) (1997)    0.325085
Dances with Wolves (1990)                     0.322470
Glory (1989)                                  0.314828
dtype: float64

-------------------------


# User-Based Collaborative Filtering


## Identifying Other Users Watching the Same Movies

In [26]:
def user_based_recommender(random_user,user_movie_df,ratio = 0.90, cor_th =0.65, score=3.5):
    
    random_user_df = user_movie_df[user_movie_df.index == random_user]
    movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
    
    movies_watched_df = user_movie_df[movies_watched]
    
    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count= user_movie_count.reset_index()
    user_movie_count.columns = ["userId","movie_count"]
    
    percentage = len(movies_watched) * ratio
    
    users_with_same_movies = user_movie_count[user_movie_count["movie_count"] > percentage]["userId"]

    final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_with_same_movies)],
                         random_user_df[movies_watched]])
    
    correlation_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    correlation_df = pd.DataFrame(correlation_df,columns=["corr"])
    correlation_df.index.names = ["user_id_1","user_id_2"]
    correlation_df = correlation_df.reset_index()
    
    top_users = correlation_df[(correlation_df["user_id_1"] == random_user) & (correlation_df["corr"] >= cor_th)][["user_id_2","corr"]].reset_index(drop=True)
        
    top_users = top_users.sort_values(by="corr",ascending=False)
    top_users.rename(columns={"user_id_2":"userId"},inplace=True)
    
    rating = pd.read_csv("datas/rating.csv")
    top_users_ratings = top_users.merge(rating[["userId","movieId","rating"]], how="inner")
    top_users_ratings["weighted_rating"] = top_users_ratings["corr"] * top_users_ratings["rating"]
    
    recommendation_df = top_users_ratings.groupby("movieId").agg({"weighted_rating":"mean"})
    recommendation_df = recommendation_df.reset_index()
    
    movies_to_be_recommended = recommendation_df[recommendation_df["weighted_rating"] > score].sort_values("weighted_rating",
                                                                                                           ascending=False)
    
    movie = pd.read_csv("datas/movie.csv")
    
    return movies_to_be_recommended.merge(movie[["movieId","title"]])

In [27]:
random_user = int(pd.Series(user_movie_df.index).sample(1).values)
user_based_recommender(random_user,user_movie_df)

,movieId,weighted_rating,title
0,79357,4.353882,Mr. Nobody (2009)
1,31660,4.353882,Steamboy (Suchîmubôi) (2004)
2,3635,4.353882,"Spy Who Loved Me, The (1977)"
3,53956,4.353882,Death at a Funeral (2007)
4,4029,4.353882,State and Main (2000)
...,...,...,...
307,6323,3.508929,Identity (2003)
308,745,3.505564,Wallace & Gromit: A Close Shave (1995)
309,1222,3.503559,Full Metal Jacket (1987)
310,30707,3.502477,Million Dollar Baby (2004)
